# ETL archivos Yelp
El objetivo es poder entender la naturaleza, formato y estructura que conteinen los archivos de Yelp para luego realizar el proceso de ETL.

* business.pkl
* review.json
* user.parquet
* checkin.json
* tip.json


In [2]:
import pandas as pd
import numpy as np
import requests
import glob
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
import pyarrow.parquet as pq
import json
import pickle

## checkin

In [3]:
def abrir_Archivo_json(archivo):
    merged_data = []  # Lista para almacenar los objetos JSON combinados

    with open(archivo) as file:
        for line in file:
            try:
                obj = json.loads(line)
                merged_data.append(obj)
            except json.JSONDecodeError as e:
                print(f"Error al decodificar JSON en el archivo {archivo}: {str(e)}")

    df = pd.DataFrame(merged_data)  # Crear DataFrame a partir de los objetos JSON
    return df


In [4]:
archivo = 'data/Yelp/checkin.json' 
df_checkin_yelp = abrir_Archivo_json(archivo)

In [5]:
#Revisamos si tiene nulos
df_checkin_yelp.isnull().sum()

business_id    0
date           0
dtype: int64

In [6]:
#Revisamos si hay duplicados
df_checkin_yelp.duplicated().sum()

0

Desanidado de Fecha y Hora

In [7]:
fecha = []
for _, row in df_checkin_yelp.iterrows():
    business_id = row['business_id']
    dates = row['date'].split(', ')
    for date in dates:
        time, date = date.split(' ')
        fecha.append([business_id, time, date])

df_desanidado = pd.DataFrame(fecha,columns =['business_id','hour','date'])

In [8]:
df_desanidado

,business_id,hour,date
0,---kPU91CF4Lq2-WlRu9Lw,2020-03-13,21:10:56
1,---kPU91CF4Lq2-WlRu9Lw,2020-06-02,22:18:06
2,---kPU91CF4Lq2-WlRu9Lw,2020-07-24,22:42:27
3,---kPU91CF4Lq2-WlRu9Lw,2020-10-24,21:36:13
4,---kPU91CF4Lq2-WlRu9Lw,2020-12-09,21:23:33
...,...,...,...
13356870,zzu6_r3DxBJuXcjnOYVdTw,2013-12-11,00:52:49
13356871,zzu6_r3DxBJuXcjnOYVdTw,2013-12-13,00:58:14
13356872,zzw66H6hVjXQEt0Js3Mo4A,2016-12-03,23:33:26
13356873,zzw66H6hVjXQEt0Js3Mo4A,2018-12-02,19:08:45


In [9]:
#Se almacena en un csv
df_desanidado.to_csv('data/Yelp/checkin.csv',index = False)

## user.parquet

## business.pkl

In [10]:
#se lee el archivo
df_business= pd.read_pickle('data/Yelp/business.pkl')

In [11]:
df_business

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,IN,T6J 5H2,53.468419,-113.492054,3.0,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,DE,37204,36.115118,-86.766925,4.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,AB,46250,39.908707,-86.065088,3.5,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,AB,62025,38.782351,-89.950558,4.0,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#Se agregan nuevas columnas en el Dataframe Vacias
df_business['NAME']=None
df_business['REVIEW_COUNT']=None
df_business['POSTAL_CODE']=None
df_business['CITY']=None
df_business['STATE']=None
df_business['BUSINESS_ID']=None
df_business['ADDRESS']=None
df_business['LATITUDE']=None
df_business['LONGITUDE']=None
df_business['STARS']=None
df_business['IS_OPEN']=None
df_business['ATTRIBUTES']=None
df_business['CATEGORIES']=None
df_business['HOURS']=None

In [13]:
#Se actualiza la columna NAME del Dataframe donde se fusionan las cadenas de texto, eliminando los caracteres no Alfabeticos
for index,i in enumerate(df_business.name.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['NAME'][index]=''.join(arr)


In [14]:

# Configurar pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)


In [15]:
#Se actualiza la columna CITY del Dataframe donde se fusionan las cadenas de texto, eliminando los caracteres no Alfabeticos
for index,i in enumerate(df_business.city.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['CITY'][index]=''.join(arr)

In [16]:
#Se actualiza la columna STATE del Dataframe donde se fusionan las cadenas de texto, eliminando los caracteres no Alfabeticos
for index,i in enumerate(df_business.state.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['STATE'][index]=''.join(arr)


In [17]:
#Se actualiza la columna REVIEW_COUNT del Dataframe donde se fusionan las cadenas de texto, eliminando los caracteres no Alfabeticos
for index,i in enumerate(df_business.state.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['REVIEW_COUNT'][index]=''.join(arr)


In [18]:
#Se actualiza la columna BUSINESS_ID del Dataframe donde se fusionan las cadenas de texto, eliminando los caracteres no Alfabeticos
for index,i in enumerate(df_business.state.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['BUSINESS_ID'][index]=''.join(arr)


In [19]:
#Se actualiza la columna ADDRESS del Dataframe donde se fusionan las cadenas de texto, eliminando los caracteres no Alfabeticos
for index,i in enumerate(df_business.state.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['ADDRESS'][index]=''.join(arr)


In [20]:
#Se actualiza la columna POSTAL_CODE del Dataframe donde se fusionan las cadenas de texto, eliminando los caracteres no Alfabeticos
for index,i in enumerate(df_business.state.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['POSTAL_CODE'][index]=''.join(arr)

In [21]:
#Se filtran los elementos numéricos mayores que 1 en cada valor de la columna 'latitude' del DataFrame 'df_business'.
# A continuación, se asigna el primer elemento filtrado a la columna 'LATITUDE' en el DataFrame 'df_business' en la fila
#correspondiente al índice actual.
for index,i in enumerate(df_business.latitude.values):
    arr=[]
    for e in i:
       if e>1:
         arr.append(e)
    df_business['LATITUDE'][index]=arr[0]


In [22]:
#Se filtran los elementos numéricos menores que -1 en cada valor de la columna 'longitude' del DataFrame 'df_business'.
#Luego, se asigna el primer elemento filtrado a la columna 'LONGITUDE' en el DataFrame 'df_business' en la fila correspondiente al
# índice actual.
for index,i in enumerate(df_business.longitude.values):
    arr=[]
    for e in i:
        if e<-1:
            arr.append(e)
    df_business['LONGITUDE'][index]=arr[0]

In [23]:
#Se filtran los elementos numéricos mayores que 0.1 en cada valor de la columna 'stars' del DataFrame 'df_business'.
# Después, se asigna el primer elemento filtrado a la columna 'STARS' en el DataFrame 'df_business' en la fila correspondiente 
#al índice actual.
for index,i in enumerate(df_business.stars.values):
    arr=[]
    for e in i:
       if e>0.1:
         arr.append(e)
    df_business['STARS'][index]=arr[0]


In [24]:
#Se filtran los elementos numéricos mayores o iguales a 0 en cada valor de la columna 'is_open' del DataFrame 'df_business'. 
#A continuación, se asigna el primer elemento filtrado a la columna 'IS_OPEN' en el DataFrame 'df_business' en la fila correspondiente al índice actual

for index,i in enumerate(df_business.is_open.values):
    arr=[]
    for e in i:
       if e >=0:
         arr.append(e)
    df_business['IS_OPEN'][index]=arr[0]

In [25]:
#Este código busca y guarda el primer diccionario encontrado en la columna 'attributes' del dataframe 'df_business' en la columna 'ATTRIBUTES'.
for index,i in enumerate(df_business.attributes.values):
    arr=[]
    for e in i:
        if isinstance(e,dict):
         arr.append(e)
    if len(arr)>0:
     df_business['ATTRIBUTES'][index]=arr[0]


In [26]:
#Este código filtra los elementos de tipo cadena en la columna 'categories' del dataframe 'df_business'. 
#Luego, fusiona todas las cadenas filtradas en una sola cadena y la asigna a la columna 'CATEGORIES' en el dataframe 'df_business'.
for index,i in enumerate(df_business.categories.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['CATEGORIES'][index]=''.join(arr)

In [27]:
#En este código, se recorren los valores de la columna 'hours' en el dataframe 'df_business' y se filtran los elementos que son diccionarios. 
#Luego, se asigna el primer diccionario encontrado a la columna 'HOURS' en el dataframe 'df_business' en la fila correspondiente.
#En resumen, el código extrae y guarda el primer diccionario encontrado en la columna 'hours' del dataframe 'df_business' en la columna 'HOURS'.
for index,i in enumerate(df_business.hours.values):
    arr=[]
    for e in i:
        if isinstance(e,dict):
         arr.append(e)
    if len(arr)>0:     
     df_business['HOURS'][index]=arr[0]


In [28]:
#Filtramos el dataframe solo a las columnas nuevas generadas con los bucles, con los nombres iniciales


In [29]:
'''
df_business = df_business[['BUSINESS_ID', 'NAME', 'REVIEW_COUNT', 'CITY', 'STATE', 'ADDRESS', 'POSTAL_CODE', 'LATITUDE', 'LONGITUDE', 'STARS', 'IS_OPEN', 'ATTRIBUTES', 'CATEGORIES', 'HOURS']]

df_business.loc[:, 'business_id'] = df_business['BUSINESS_ID']
df_business.loc[:, 'name'] = df_business['NAME']
df_business.loc[:, 'address'] = df_business['ADDRESS']
df_business.loc[:, 'city'] = df_business['CITY']
df_business.loc[:, 'state'] = df_business['STATE']
df_business.loc[:, 'postal_code'] = df_business['POSTAL_CODE']
df_business.loc[:, 'latitude'] = df_business['LATITUDE']
df_business.loc[:, 'longitude'] = df_business['LONGITUDE']
df_business.loc[:, 'stars'] = df_business['STARS']
df_business.loc[:, 'review_count'] = df_business['REVIEW_COUNT']
df_business.loc[:, 'is_open'] = df_business['IS_OPEN']
df_business.loc[:, 'attributes'] = df_business['ATTRIBUTES']
df_business.loc[:, 'categories'] = df_business['CATEGORIES']
df_business.loc[:, 'hours'] = df_business['HOURS']

df_business = df_business[['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours']]
'''


"\ndf_business = df_business[['BUSINESS_ID', 'NAME', 'REVIEW_COUNT', 'CITY', 'STATE', 'ADDRESS', 'POSTAL_CODE', 'LATITUDE', 'LONGITUDE', 'STARS', 'IS_OPEN', 'ATTRIBUTES', 'CATEGORIES', 'HOURS']]\n\ndf_business.loc[:, 'business_id'] = df_business['BUSINESS_ID']\ndf_business.loc[:, 'name'] = df_business['NAME']\ndf_business.loc[:, 'address'] = df_business['ADDRESS']\ndf_business.loc[:, 'city'] = df_business['CITY']\ndf_business.loc[:, 'state'] = df_business['STATE']\ndf_business.loc[:, 'postal_code'] = df_business['POSTAL_CODE']\ndf_business.loc[:, 'latitude'] = df_business['LATITUDE']\ndf_business.loc[:, 'longitude'] = df_business['LONGITUDE']\ndf_business.loc[:, 'stars'] = df_business['STARS']\ndf_business.loc[:, 'review_count'] = df_business['REVIEW_COUNT']\ndf_business.loc[:, 'is_open'] = df_business['IS_OPEN']\ndf_business.loc[:, 'attributes'] = df_business['ATTRIBUTES']\ndf_business.loc[:, 'categories'] = df_business['CATEGORIES']\ndf_business.loc[:, 'hours'] = df_business['HOURS']

## tip.json

In [31]:
json_objects=[]

with open('data/Yelp/tip.json', 'r',encoding='utf-8') as f:
    for line in f:
        json_objects.append(json.loads(line))
df_tip = pd.DataFrame(json_objects)

In [33]:
df_tip

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0
...,...,...,...,...,...
908910,eYodOTF8pkqKPzHkcxZs-Q,3lHTewuKFt5IImbXJoFeDQ,Disappointed in one of your managers.,2021-09-11 19:18:57,0
908911,1uxtQAuJ2T5Xwa_wp7kUnA,OaGf0Dp56ARhQwIDT90w_g,Great food and service.,2021-10-30 11:54:36,0
908912,v48Spe6WEpqehsF2xQADpg,hYnMeAO77RGyTtIzUSKYzQ,Love their Cubans!!,2021-11-05 13:18:56,0
908913,ckqKGM2hl7I9Chp5IpAhkw,s2eyoTuJrcP7I_XyjdhUHQ,Great pizza great price,2021-11-20 16:11:44,0


In [32]:
df_tip=df_tip[df_tip['business_id'].isin(BusinessYelp.business_id.unique().tolist())]

NameError: name 'BusinessYelp' is not defined